In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os

In [2]:
data_frame = pd.read_csv(os.path.join("../","dataset/Tweets.csv"))

In [3]:
data_frame.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [4]:
data_frame.drop(columns=['tweet_id','airline_sentiment_confidence','tweet_coord',	'tweet_created','tweet_location','user_timezone','name','negativereason_gold','retweet_count','negativereason','negativereason_confidence','airline'],inplace=True)

In [5]:
data_frame.drop(columns=['airline_sentiment_gold'], inplace=True)

In [6]:
data_frame.isnull().sum()

airline_sentiment    0
text                 0
dtype: int64

In [7]:
data_frame.head()

,airline_sentiment,text
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...
2,neutral,@VirginAmerica I didn't today... Must mean I n...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...


In [9]:
def remove_tags(text):
    tokens = text.split()
    filtered = [tok for tok in tokens if not tok.startswith('@')]
    return ' '.join(filtered)


In [10]:
data_frame['clean_text']=data_frame['text'].apply(remove_tags)

In [11]:
data_frame.head()

,airline_sentiment,text,clean_text
0,neutral,@VirginAmerica What @dhepburn said.,What said.
1,positive,@VirginAmerica plus you've added commercials t...,plus you've added commercials to the experienc...
2,neutral,@VirginAmerica I didn't today... Must mean I n...,I didn't today... Must mean I need to take ano...
3,negative,@VirginAmerica it's really aggressive to blast...,"it's really aggressive to blast obnoxious ""ent..."
4,negative,@VirginAmerica and it's a really big bad thing...,and it's a really big bad thing about it


In [69]:
counts = data_frame['airline_sentiment'].value_counts()
print(counts)

airline_sentiment
negative    9178
neutral     3099
positive    2363
Name: count, dtype: int64


In [12]:
vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=20_000,        
    output_mode='int',         
    output_sequence_length=256 
)
texts = data_frame['clean_text'].astype(str).to_numpy()  
vectorizer.adapt(texts) 
tokenized_tensor = vectorizer(texts)

In [17]:
from  tensorflow.keras.layers import Dense, GlobalAveragePooling1D, Input,Embedding
from tensorflow.keras.models import Model

In [28]:
max_length = 256
embbedding_dim = 16
vocal_size = 20_000
input = Input(shape=(max_length,))
x = Embedding(vocal_size, embbedding_dim)(input)
x = Dense(units=256, activation='relu')(x)
x = GlobalAveragePooling1D()(x)
output_layer= Dense(units=3, activation='softmax')(x)
model = Model(inputs=input, outputs =output_layer)

model.summary()






Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_7 (InputLayer)      │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_4 (Embedding)         │ (None, 256, 16)        │       320,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 256, 256)       │         4,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_3      │ (None, 256)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 3)              │           771 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 325,123 (1.24 MB)

 Trainable params: 325,123 (1.24 MB)

 Non-trainable params: 0 (0.00 B)